## 2.4 Model Selection 소개
### 2.4.1 학습/테스트 데이터 셋 분리 – train_test_split()

* "왜 학습데이터와 테스트데이터를 분리해야할까?"

iris 데이터를 나누지 말고 전체 데이터로 모델을 만든 다음, 역시 같은 데이터로 모델의 성능 검증까지 해보자.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target

In [3]:
dt_clf.fit(train_data, train_label)
# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:', accuracy_score(train_label,pred))

예측 정확도: 1.0


정확도가 100%다. 완벽하게 예측에 성공했으므로 좋은 결과일까? 전혀 아니다. 동일한 데이터로 학습과 검증을 모두 한다면 중간고사를 보고 나서 답을 다 알려준 다음 다시 꼭같은 문제로 기말고사를 보는 것과 같으므로 머신러닝 모델의 성능을 제대로 알 수 없다. 따라서 학습하기 전에 미리 `sklearn의model_selection` 안에 있는 `train_test_split()`을 이용해서 데이터를 분리해야한다.

In [4]:
from sklearn.model_selection import train_test_split

train_test_split()의 괄호 속에 들어가는 parameter들에 대해서는 101쪽의 설명을 보라. 

train_test_split()의 실행결과는 학습과 테스트용 피쳐 데이터, 그리고 학습과 테스트용 레이블 데이터로 이루어진 튜플이다.

In [5]:
dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)

In [6]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


### 2.4.2 교차 검증

**overfitting(과대적합)**

앞에서 우리는 학습(훈련)데이터와 테스트데이터를 분리해서 학습과 성능 평가를 해야한다는 것을 알았다. 전체 데이터를 둘로 나눈 다음 학습과 테스트를 거쳐서 정확도가 높은 머신러닝 모형을 만들었다고 해보자. 그런데 우리가 머신러닝 모델을 만드는 목적이 학습과 테스트에 이용한 데이터에만 적용하려는 것은 아니다. 이제 기존 데이터가 아닌 새로운 데이터가 더 있다고 해보자. 우리의 모델이 과연 그 새로운 데이터에 대해서도 좋은 성능을 보일까? 만약 그렇지 않고 모델을 만들고 테스트한 데이터에서는 높은 성능을 보이지만 새로운 데이터에서는 성능이 떨어진다면 그 모델은 좋은 모델이 아닐 것이다. 

이럴 때 그 모델은 <u>'과대적합(과적합, overfitting)'</u>이라고 불리는 문제를 갖는다고 말한다. 즉 `과대적합이란 사소한 잡음을 비롯하여 데이터에 있는 중요하지 않은 디테일까지 너무 세밀하게 살피는 머신러닝 모델을 만든 다음, 그 모델이 다른 데이터에 대해서도 좋은 성능을 보일 것이라고 과대평가하는 위험을 말한다.` 시험에 대비해서 교과서 내용을 빠짐없이 모두 외웠는데 시험이 항상 교과서 그대로 나오지는 않는다. 나름 열심히 공부했지만 막상 실제 시험에는 교과서 내용을 조금씩 응용한 문제가 나오는 바람에 못 풀게되는 경우를 생각하면 되겠다. 

**검증데이터셋**

과대적합 문제를 피하기 위해 흔히 쓰는 방법이 `검증(validation)`이다. 지금까지는 데이터를 train data, test data 둘로 나누었지만 검증을 하기 위해서는 `검증데이터(validation data)`도 필요하다. 검증을 위해서는 일반적으로 전체 데이터를 먼저 학습(train)데이터와 테스트데이터(hold out test data)로 나눈 다음 학습데이터를 또 학습, 검증데이터로 나누어서 모델을 1차 평가한다. 그런 과정을 거친 모델에 대해 테스트데이터로 모델의 성능을 최종 평가한다. 가령 전체 데이터를 먼저 75:25로 나누어서 25%는 최종 hold out test data로 이용하고 75%를 다시 나누어 50%는 훈련, 25%는 검증에 사용하는 식이다.

**K 폴드 교차검증(K fold cross-validation)**

**(KFold, StratifiedKFold, cross_cal_score)**

위의 설명에 따르면 학습, 검증데이터는 서로 겹치지 않아야할 것 같다. 하지만 그런 방법은 데이터가 많지 않으면 이용하기 어렵다. 따라서 일반적으로 널리 쓰이는 검증 방법은 <u>'교차검증'</u>, 즉 데이터를 크기가 같은 여러 부분(K 개의 폴드(fold) 세트)으로 나눈 다음 `K-1개 폴드 세트로는 학습`을 하고 `나머지 한 개 폴드로 검증`하는 과정을 `K번 반복`하는 것이다. 즉 교차검증에서는 검증 데이터셋을 따로 만들지 않으므로 데이터들이 서로 교차되면서 학습에도 쓰이고 검증에도 쓰이게 된다. `교재 104쪽` 그림을 보자. 이런 과정을 통해 머신러닝 모델을 K번 검증 평가하게 되고 마지막에 그 평가 결과를 평균해서 모델에 대한 최종 교차평가를 수행한다. 

* 사이킷런을 이용한 K-fold 교차검증 (KFold, StratifiedKFold)

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [8]:
iris = load_iris()
features = iris.data
label = iris.target
print(features.shape, label.shape)

(150, 4) (150,)


In [9]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [10]:
dt_clf = DecisionTreeClassifier(random_state=156)
kfold = KFold(5)
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

KFold에서 K값, 즉 n_splits의 `default 값은 5`다. 위의 코드에서는 클래스 KFold를 이용하여 객체 kfold를 만들었다. kfold 객체의 split()메서드의 결과는 학습데이터와 검증데이터로 분할하는 index다. 아래 결과를 보자

In [12]:
for train_index, test_index in kfold.split(features):
    print(len(train_index), len(test_index))

120 30
120 30
120 30
120 30
120 30


분할 수가 5인 KFold (5-fold cv)이므로 150개 데이터를 4:1로, 즉 120:30으로 나누어서 훈련, 검증데이터로 이용한다. 각 index를 보자.

In [13]:
for train_index, test_index in kfold.split(features):
    print(train_index, test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

index 번호를 보면 데이터를 전혀 섞지 않고 번갈아 가며 4:1로 나눈 것을 확인할 수 있다.

이제 위의 index에 해당하는 X, y 데이터를 train, test data로 나누자.

In [14]:
# 5개의 폴드 세트로 분리하는 KFold 객체(kfold)와 폴드 세트별 정확도를 담을 리스트 객체(cv_accuracy) 생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])

붓꽃 데이터 세트 크기: 150


 KFold에서 K값, 즉 n_splits의 default 값은 5다.

In [16]:
kfold = KFold(5)

n_iter = 0 # 파라미터 검색 횟수
cv_accuracy = []
# KFold객체의 split( ) 호출하면 폴드 별 학습용 data와 검증용 data의 row index를 array로 반환  
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1 # a += b : a = a + b
    # 반복 시 마다 정확도 측정 , round -반올림
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy) # 선택된 요소의 마지막에 새로운 HTML 요소나 콘텐츠를 추가한다.
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print()
print(cv_accuracy)
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

[1.0, 0.9667, 0.8667, 0.9333, 0.7333]

## 평균 검증 정확도: 0.9


- **출석확인문제**

위의 코드에서 for문의 반복횟수는?

In [21]:
print(n_iter)

5


이번에는 3-fold cv를 연습해보자. 150개 데이터를 2:1 (100:50)로 나누너 번갈아 검증한다.

출석확인문제 : 정확도가 모두 0이 되었다! 전혀 분류하지 못했다는 뜻이다. 왜 그런가?

150개의 데이터를 계층별로 분류교차검증을 하는데,
1차 교차 검증에서는 0 ~ 49번까지, 두번째는 50 ~ 99번까지, 세번째는 100 ~ 149번으로 
각 50개의 검증 세트를 인덱스를 생성하였는데 
이렇게 되면 `학습용 데이터와 검증데이터가 겹치지 않아 검증하는것이 불가능하기 때문이다.`

In [22]:
kfold = KFold(3)

n_iter = 0
cv_accuracy = []
# KFold객체의 split( ) 호출하면 폴드 별 학습용 data와 검증용 data의 row index를 array로 반환  
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시 마다 정확도 측정 , round -반올림
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print()
print(cv_accuracy)
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

#2 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]

#3 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

[0.0, 0.0, 0.0]

## 평균 검증 정확도: 0.0


* **Stratified K 폴드**

    만일 금융기관의 대출 거래 기록 데이터에서 label이 정상 대출, 대출 사기 여부라고 해보자. 대부분의 거래는 정상 거래이고 사기 거래는 아주 드물 것이다. 이럴 경우 전체 데이터를 KFold를 이용해서 K개의 폴드로 나누면 어떤 폴드에는 정상 대출 데이터만 포함되고 대출 사기 데이터가 거의 없는 경우도 생길 것이다. 층화 폴드(stratified K fold)는 각 폴드의 `label 분포가 원본 데이터의 label 분포와 동일하게 만들어 준다.` 층화 방법은 `질병 발생 여부, 복권 당첨 여부 등 발생 빈도가 낮은 사건을 나타내는 데이터에서 특히 유용하다.` 교재 106-107쪽 설명을 읽어보자.

In [23]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

붓꽃의 종류가 3개인데 데이터 수는 각각 50개씩이다.

아래와 같이 `K=3`으로 두고 KFold로 세 폴드를 만들면 어떤 일이 생길까? 아래 결과를 보자.

In [24]:
kfold = KFold(n_splits=3)
# kfold.split()는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터의 row index 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    print()   

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64

## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64

## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64



첫번째 교차 검증에서는 붓꽃 종류 1,2가 학습데이터가 되고 종류 0이 검증데이터가 되었다. 즉 종류 0은 학습될 수 없으므로 검증 결과 정확도가 0이 될 것이다. 두번째와 세번째 교차 검증에서도 마찬가지다. 이제 sklearn.model_selection 모듈에서 `StratifiedKFold`를 부른 다음, 층화 K-fold를 이용해보자. 각 폴드마다 0,1,2 세 종류의 붓꽃 데이터가 학습, 검증 데이터에 고르게 포함된 것을 확인할 수 있다.

In [28]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    print() 

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64

## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64

## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64



In [29]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


* **cross_val_score( )**

위와 같이 하는 대신 sklearn.model_selection의 cross_val_score( )를 이용하면 교차 검증 과정이 간단해진다. 즉 아래 과정들을 한꺼번에 편리하게 수행할 수 있다. 

 `1) 폴더 세트 설정, 2) for 문을 이용한 학습, 3) 검증 데이터 index 추출, 반복 학습과 예측 수행.`

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy', cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


cross_validate()도 비슷한 역할을 하는데, 단 하나의 평가지표(scoring =)만 지정하는 cross_val_score()와 달리 여러 평가지표를 지정할 수 있다. 

### 2.4.3 GridSearchCV - 교차검증과 최적 hyper parameter tuning을 한 번에

**hyper parameter와 GridSearchCV**  

머신러닝 모델 평가 과정 안에는 모델 학습 단계와 모델 선택 단계가 포함된다. 모델 선택 단계에서는 이용할 모델과 hyper parameter를 선택한다. 이때 이용하는 데이터는 검증데이터다. 그 다음에 선택한 학습데이터를 이용하여 모델 안에서 parameter를 조정한다. 

교재에서 iris 데이터를 분류하기 위해 이용하는 의사결정나무 분류기(DecisionTreeClassifier)에서 필요로하는 hyper parameter로는 max_depth, min_sample_split등이 있다. 여기서 `max_depth`는 분류 가지를 몇 단계까지 나눌 것인가를 나타내고 `min_sample_split`은 node를 더 이상 나누지 않는 최소 데이터 수를 말한다. 즉 `min_sample_split가 클수록, 그리고 max_depth가 작을수록 나무가 간단해진다. `

이제 교재 113-114쪽처럼 max_depth 값을 1,2,3, 그리고 min_sample_split 값을 2,3 중에서 고른다고 해보자. 즉 가능한 여섯 가지 hyper parameter의 조합에 대해 학습한 다음 최적의 hyper parameter 조합을 찾아야한다. 이럴 때 **GridSearchCV**으로 교차검증 방법에 의해 최적값을 찾을할 수 있다. GridSearchCV 역시 `sklearn.model_selection` 안에 들어있다.

In [17]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [18]:
# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.  
# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)
grid_dtree.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

GridSearchCV 결과는 `cv_results_` 라는 dictionary에 들어있으며 필요한 것들을 추출하여 DataFrame으로 변환할 수 있다.

In [19]:
sorted(grid_dtree.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_max_depth',
 'param_min_samples_split',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [34]:
import pandas as pd
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


또한\
`best_estimator_`\
`best_score_`\
`best_params_` 등의 결과도 볼 수 있다.

In [35]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750


In [36]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667


결과에 대한 설명은 교재 116-7쪽 참조.

- 출석퀴즈

In [5]:
f = lambda x : len(x) 
x = ['a', 1, (), [99, 12]] 
d = {'이름' : '다미', '학번' : 531022, '학과' : '빅데이터응용통계'}

In [6]:
f('kyungsung')

9

In [7]:
f(x)

4

In [8]:
f(d)

3

scikit-learn의 model_selection 모듈에 들어있지 않은 것은? *

In [ ]:
train_test_split
feature_selection *
cross_val_score 
GridSearchCV

- 출석퀴즈

밑의 코드 출력결과는? *

In [1]:
import numpy as np
x = np.arange(4)
y = x.reshape(1, -1)
print(y)

[[0 1 2 3]]


다음 중 최적 parameter를 선택하기 위한 것은? *

In [ ]:
GridSearchCV() *
StratifiedKFold()
cross_val_score()
KFold()

특정 값에 치우친 label 값을 가진 데이터에 적절한 방법은? *

In [ ]:
cross_val_score()
StratifiedKFold() *
GridSearchCV()
OneHotEncoder()

import numpy as np

x = np.arange(6)

y =

print(y)

[[0 1]
 [2 3]
 [4 5]]
 
다음과 같이 나오려면 y에 들어갈 값은?

In [1]:
import numpy as np
x = np.arange(6)
y = x.reshape(3, 2)
print(y) 

[[0 1]
 [2 3]
 [4 5]]
